In [1]:
import boto3
import pandas as pd
from io import StringIO
import time

In [2]:
AWS_ACCESS_KEY=''
AWS_SECRET_KEY=''
AWS_REGION='ap-south-1'
SCHEMA_NAME='covid'
S3_STAGING_DIR=''
S3_BUCKET_NAME='athena-outputqueries'
S3_OUTPUT_DIRECTORY='output'

In [3]:
athena_client=boto3.client(
"athena",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION
)

In [4]:
Dict={}
def download_and_load_query_results(client: boto3.client, query_response: Dict) -> pd.DataFrame:
    while True:
        try:
            
            client.get_query_results(
            QueryExecutionId=query_response['QueryExecutionId']
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
    temp_file_location: str= "athena_query_results.csv"
    s3_client=boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
        
    )
    s3_client.download_file(
    S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    
    return pd.read_csv(temp_file_location)

In [5]:
response= athena_client.start_query_execution(
    QueryString="SELECT * FROM covid_data;",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    },
)

In [6]:
response

{'QueryExecutionId': '1dfe6d21-9e5b-4794-a09f-da6d42f73597',
 'ResponseMetadata': {'RequestId': '4ba2368e-e8e8-461c-b7b3-25706231dc98',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 05 Jan 2023 07:42:29 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4ba2368e-e8e8-461c-b7b3-25706231dc98'},
  'RetryAttempts': 0}}

In [7]:
covidData=download_and_load_query_results(athena_client, response)

In [8]:
covidData.head()

,updatedon,state,totaldosesadministered,sessions,sites,firstdoseadministered,seconddoseadministered,male(dosesadministered),female(dosesadministered),transgender(dosesadministered),...,18-44years(dosesadministered),45-60years(dosesadministered),60+years(dosesadministered),18-44years(individualsvaccinated),45-60years(individualsvaccinated),60+years(individualsvaccinated),male(individualsvaccinated),female(individualsvaccinated),transgender(individualsvaccinated),totalindividualsvaccinated
0,16/01/2021,India,48276.0,3455.0,2957.0,48276.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23757.0,24517.0,2.0,48276.0
1,17/01/2021,India,58604.0,8532.0,4954.0,58604.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,27348.0,31252.0,4.0,58604.0
2,18/01/2021,India,99449.0,13611.0,6583.0,99449.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,41361.0,58083.0,5.0,99449.0
3,19/01/2021,India,195525.0,17855.0,7951.0,195525.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,81901.0,113613.0,11.0,195525.0
4,20/01/2021,India,251280.0,25472.0,10504.0,251280.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,98111.0,153145.0,24.0,251280.0


In [9]:
covidData=covidData.sort_values(by='updatedon')

In [65]:
covidData.groupby(['state']).mean()


,totaldosesadministered,sessions,sites,firstdoseadministered,seconddoseadministered,male(dosesadministered),female(dosesadministered),transgender(dosesadministered),covaxin(dosesadministered),covishield(dosesadministered),...,18-44years(dosesadministered),45-60years(dosesadministered),60+years(dosesadministered),18-44years(individualsvaccinated),45-60years(individualsvaccinated),60+years(individualsvaccinated),male(individualsvaccinated),female(individualsvaccinated),transgender(individualsvaccinated),totalindividualsvaccinated
state,,,,,,,,,,,,,,,,,,,,,
Andaman and Nicobar Islands,9.973014e+04,3.550427e+03,20.990291,7.973716e+04,1.999298e+04,5.074636e+04,4.351929e+04,9.349515,0.000000e+00,9.957004e+04,...,8.931333e+04,1.038225e+05,5.291907e+04,1.211212e+04,4.333205e+04,2.221060e+04,NaN,NaN,NaN,5.063828e+04
Andhra Pradesh,7.726594e+06,3.419349e+05,1728.276699,5.984761e+06,1.741833e+06,3.289657e+06,3.665310e+06,1033.058252,1.280074e+06,6.427122e+06,...,5.179065e+06,8.652103e+06,5.045893e+06,1.000233e+06,2.516865e+06,1.860175e+06,NaN,NaN,NaN,3.528694e+06
Arunachal Pradesh,2.958121e+05,1.372795e+04,112.961165,2.378883e+05,5.792387e+04,1.486204e+05,1.195919e+05,84.199029,8.834951e+00,2.951482e+05,...,4.288243e+05,2.259950e+05,8.838776e+04,7.386651e+04,8.895830e+04,3.530951e+04,NaN,NaN,NaN,1.317597e+05
Assam,3.477617e+06,1.712296e+05,964.208738,2.842720e+06,6.348970e+05,1.727155e+06,1.473356e+06,470.723301,5.763121e+05,2.898458e+06,...,4.427698e+06,3.152909e+06,1.541850e+06,6.078973e+05,1.068489e+06,6.061662e+05,NaN,NaN,NaN,1.498557e+06
Bihar,8.452881e+06,5.956884e+05,1850.014563,7.138363e+06,1.314517e+06,4.220362e+06,3.627255e+06,1437.640777,7.612016e+05,7.682524e+06,...,9.546678e+06,5.885181e+06,5.351997e+06,1.436820e+06,2.234556e+06,2.622210e+06,NaN,NaN,NaN,4.130299e+06
Chandigarh,2.732856e+05,1.142685e+04,49.072816,2.170054e+05,5.628027e+04,1.417855e+05,1.076082e+05,32.033981,5.712524e+02,2.725500e+05,...,3.355703e+05,2.137306e+05,1.387471e+05,5.235367e+04,7.835130e+04,5.765754e+04,NaN,NaN,NaN,1.233219e+05
Chhattisgarh,4.699628e+06,5.885324e+05,1977.893204,3.864092e+06,8.355359e+05,2.127015e+06,2.188191e+06,469.121359,2.782018e+05,4.415312e+06,...,3.171658e+06,4.199207e+06,2.492610e+06,4.131424e+05,2.270624e+06,1.467081e+06,NaN,NaN,NaN,2.720683e+06
Dadra and Nagar Haveli and Daman and Diu,1.862901e+05,1.382505e+04,53.965854,1.638784e+05,2.241179e+04,1.160419e+05,6.165029e+04,32.980488,5.229268e+00,1.862089e+05,...,3.847275e+05,1.196624e+05,4.072009e+04,5.157748e+04,4.145143e+04,1.754824e+04,NaN,NaN,NaN,7.143774e+04
Delhi,3.944458e+06,1.065233e+05,606.422330,3.030774e+06,9.136842e+05,2.057803e+06,1.481321e+06,720.922330,1.061612e+06,2.874121e+06,...,4.409175e+06,2.969702e+06,1.827012e+06,9.005017e+05,1.181362e+06,8.150006e+05,NaN,NaN,NaN,1.910858e+06


In [11]:
covidData=covidData.reset_index()



In [12]:
covidData=covidData.drop(['index'], axis=1)

In [13]:
covidData

,updatedon,state,totaldosesadministered,sessions,sites,firstdoseadministered,seconddoseadministered,male(dosesadministered),female(dosesadministered),transgender(dosesadministered),...,18-44years(dosesadministered),45-60years(dosesadministered),60+years(dosesadministered),18-44years(individualsvaccinated),45-60years(individualsvaccinated),60+years(individualsvaccinated),male(individualsvaccinated),female(individualsvaccinated),transgender(individualsvaccinated),totalindividualsvaccinated
0,1/2/2021,West Bengal,238178.0,5304.0,759.0,238178.0,0.0,77261.0,160810.0,107.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238178.0
1,1/2/2021,Bihar,144845.0,4647.0,948.0,144845.0,0.0,39417.0,105337.0,91.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144845.0
2,1/2/2021,Dadra and Nagar Haveli and Daman and Diu,1038.0,35.0,19.0,1038.0,0.0,476.0,562.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1038.0
3,1/2/2021,Goa,3860.0,119.0,60.0,3860.0,0.0,1457.0,2403.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3860.0
4,1/2/2021,Puducherry,2316.0,100.0,14.0,2316.0,0.0,1087.0,1227.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2316.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7840,9/8/2021,Bihar,27834939.0,1151080.0,1096.0,23350171.0,4484768.0,14926420.0,12902990.0,5529.0,...,13984649.0,7520647.0,6329643.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7841,9/8/2021,Arunachal Pradesh,879094.0,31506.0,129.0,692475.0,186619.0,486874.0,391993.0,227.0,...,514163.0,263660.0,101271.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7842,9/8/2021,Himachal Pradesh,5632441.0,162941.0,701.0,4249849.0,1382592.0,2772475.0,2858812.0,1154.0,...,2216211.0,1970847.0,1445383.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7843,9/8/2021,Chhattisgarh,11769177.0,1390084.0,1027.0,9181482.0,2587695.0,5916437.0,5851349.0,1391.0,...,4389004.0,4705765.0,2674408.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:

covidData.head()

,updatedon,state,totaldosesadministered,sessions,sites,firstdoseadministered,seconddoseadministered,male(dosesadministered),female(dosesadministered),transgender(dosesadministered),...,18-44years(dosesadministered),45-60years(dosesadministered),60+years(dosesadministered),18-44years(individualsvaccinated),45-60years(individualsvaccinated),60+years(individualsvaccinated),male(individualsvaccinated),female(individualsvaccinated),transgender(individualsvaccinated),totalindividualsvaccinated
0,1/2/2021,West Bengal,238178.0,5304.0,759.0,238178.0,0.0,77261.0,160810.0,107.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238178.0
1,1/2/2021,Bihar,144845.0,4647.0,948.0,144845.0,0.0,39417.0,105337.0,91.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144845.0
2,1/2/2021,Dadra and Nagar Haveli and Daman and Diu,1038.0,35.0,19.0,1038.0,0.0,476.0,562.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1038.0
3,1/2/2021,Goa,3860.0,119.0,60.0,3860.0,0.0,1457.0,2403.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3860.0
4,1/2/2021,Puducherry,2316.0,100.0,14.0,2316.0,0.0,1087.0,1227.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2316.0


In [106]:
sortedcovidData=covidData.sort_values(by='state').reset_index().drop(['index'], axis=1)
sortedcovidData

,updatedon,state,totaldosesadministered,sessions,sites,firstdoseadministered,seconddoseadministered,male(dosesadministered),female(dosesadministered),transgender(dosesadministered),...,18-44years(dosesadministered),45-60years(dosesadministered),60+years(dosesadministered),18-44years(individualsvaccinated),45-60years(individualsvaccinated),60+years(individualsvaccinated),male(individualsvaccinated),female(individualsvaccinated),transgender(individualsvaccinated),totalindividualsvaccinated
0,16/05/2021,Andaman and Nicobar Islands,115188.0,2120.0,21.0,100058.0,15130.0,53903.0,46147.0,8.0,...,NaN,NaN,NaN,10447.0,59293.0,30316.0,NaN,NaN,NaN,100058.0
1,4/4/2021,Andaman and Nicobar Islands,23901.0,1200.0,12.0,18677.0,5224.0,10996.0,7681.0,0.0,...,NaN,NaN,NaN,5165.0,8580.0,4930.0,NaN,NaN,NaN,18677.0
2,25/07/2021,Andaman and Nicobar Islands,279839.0,2817.0,16.0,193281.0,86558.0,149816.0,129989.0,34.0,...,99170.0,120205.0,60464.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13/04/2021,Andaman and Nicobar Islands,48845.0,3400.0,32.0,43013.0,5832.0,23433.0,19575.0,5.0,...,NaN,NaN,NaN,5287.0,23593.0,14131.0,NaN,NaN,NaN,43013.0
4,24/03/2021,Andaman and Nicobar Islands,19693.0,1400.0,14.0,15247.0,4446.0,9159.0,6088.0,0.0,...,NaN,NaN,NaN,4936.0,7062.0,3246.0,NaN,NaN,NaN,15247.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7840,10/2/2021,West Bengal,396442.0,11924.0,973.0,396442.0,0.0,150028.0,246289.0,125.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,396442.0
7841,16/02/2021,West Bengal,528075.0,15509.0,1045.0,521537.0,6538.0,228169.0,293240.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,521537.0
7842,25/01/2021,West Bengal,87815.0,2713.0,632.0,87815.0,0.0,28568.0,59165.0,82.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87815.0
7843,15/03/2021,West Bengal,2353011.0,713800.0,3610.0,1958513.0,394498.0,1397096.0,955639.0,276.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2353011.0


In [14]:
bucket='aws-covd-project'

In [15]:
csv_buffer=StringIO()

In [16]:
csv_buffer

In [17]:
covidData.to_csv(csv_buffer)

In [18]:
s3_resource = boto3.resource('s3',
        aws_access_key_id=AWS_ACCESS_KEY,
         aws_secret_access_key= AWS_SECRET_KEY
                            )


In [ ]:
s3_resource.Object(bucket, 'output/covid19Data.csv').put(Body=csv_buffer.getvalue())

In [19]:
covidDatasql=pd.io.sql.get_schema(covidData,'covidData')
print(''.join(covidDatasql))

CREATE TABLE "covidData" (
"updatedon" TEXT,
  "state" TEXT,
  "totaldosesadministered" REAL,
  "sessions" REAL,
  "sites" REAL,
  "firstdoseadministered" REAL,
  "seconddoseadministered" REAL,
  "male(dosesadministered)" REAL,
  "female(dosesadministered)" REAL,
  "transgender(dosesadministered)" REAL,
  "covaxin(dosesadministered)" REAL,
  "covishield(dosesadministered)" REAL,
  "sputnikv(dosesadministered)" REAL,
  "aefi" REAL,
  "18-44years(dosesadministered)" REAL,
  "45-60years(dosesadministered)" REAL,
  "60+years(dosesadministered)" REAL,
  "18-44years(individualsvaccinated)" REAL,
  "45-60years(individualsvaccinated)" REAL,
  "60+years(individualsvaccinated)" REAL,
  "male(individualsvaccinated)" REAL,
  "female(individualsvaccinated)" REAL,
  "transgender(individualsvaccinated)" REAL,
  "totalindividualsvaccinated" REAL
)


In [41]:
import redshift_connector
import boto3

In [53]:
conn =redshift_connector.connect(
    host='',
    database='dev',
    user='awsuser',
    password='',
    port=5439,
    
 )

# import redshift_connector

# # Connects to Redshift cluster using IAM credentials from default profile defined in ~/.aws/credentials
# conn = redshift_connector.connect(
#     iam=True,
#     database='dev',
#     user='AwsRed',
#     db_user='awsuser',
#     password='',
#     cluster_identifier='my-first-cluster',
#     access_key_id="",
#     secret_access_key="",
#     region="ap-south-1"
#  )

InterfaceError: ('communication error', gaierror(11001, 'getaddrinfo failed'))

In [52]:
conn.autocommit=True

NameError: name 'conn' is not defined

In [ ]:
cursor=redshift_connector.Cursor = conn.cursor()

In [ ]:
cursor.execute("""
CREATE TABLE "covidData" (
"updatedon" TEXT,
  "state" TEXT,
  "totaldosesadministered" REAL,
  "sessions" REAL,
  "sites" REAL,
  "firstdoseadministered" REAL,
  "seconddoseadministered" REAL,
  "male(dosesadministered)" REAL,
  "female(dosesadministered)" REAL,
  "transgender(dosesadministered)" REAL,
  "covaxin(dosesadministered)" REAL,
  "covishield(dosesadministered)" REAL,
  "sputnikv(dosesadministered)" REAL,
  "aefi" REAL,
  "18-44years(dosesadministered)" REAL,
  "45-60years(dosesadministered)" REAL,
  "60+years(dosesadministered)" REAL,
  "18-44years(individualsvaccinated)" REAL,
  "45-60years(individualsvaccinated)" REAL,
  "60+years(individualsvaccinated)" REAL,
  "male(individualsvaccinated)" REAL,
  "female(individualsvaccinated)" REAL,
  "transgender(individualsvaccinated)" REAL,
  "totalindividualsvaccinated" REAL
)
""")

In [ ]:
cursor.execute("""
copy covidData from 's3://aws-covd-project/output/covid19Data.csv'
credentials ''
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
"""
)